In [ ]:
!pip install biopython
from Bio import SeqIO


In [ ]:
from datetime import datetime

# Proccessing data base

In [ ]:
# Run this first to Create Data Dictionaries (Habitat, Taxa, Cog_info)
# Creating Taxa Dictionary
def process_taxa_file():
    data_dict = {}
    with open(taxa_file_name, 'r') as file:
        for line in file:
            parts = line.strip().split(',')
            strain_id = parts[-3]
            bacteria_id = parts[-4]
            phylum_name = parts[1]
            list = [strain_id, phylum_name]
            if (strain_id!='strain_id'):
              data_dict[bacteria_id] = list
    return data_dict

# # Creating cog_info Dictionary
def process_cog_info_file(file_name, encoding='utf-8'):
    data_dict = {}
    with open(file_name, 'r', encoding=encoding, errors='replace') as file:
        for line_number, line in enumerate(file, start=1):
            try:
                parts = line.strip().split(';')
                if len(parts) == 6:
                    cog = parts[0]
                    key = cog[3:]
                    value = parts[1]+';'+parts[2]+';'+parts[3]+';'+parts[4]
                    data_dict[key] = value

            except UnicodeDecodeError as e:
                print(f"Error decoding line {line_number}: {e}")
                print(f"Problematic line content: {line}")
    return data_dict



# Creating Habitat Dictionary
def process_habitat_file():
    data_dict = {}
    with open(habitat_file_name, 'r') as file:
        for line in file:
            parts = line.strip().split(';')
            host_name = parts[1]
            habitat = parts[-1]
            if (host_name!='bacteria'):
              data_dict[host_name] = habitat
    return data_dict


# Main Code

taxa_file_name = 'taxa.txt'
taxa_dict = process_taxa_file()

habitat_file_name = 'habitat.txt'
habitat_dict = process_habitat_file()

cog_file_name = 'COG_INFO_TABLE.txt'
cog_dict = process_cog_info_file(cog_file_name)

# Extracring information

In [ ]:
class Genome:
    def __init__(self, Genome_id, creature_name):
        self.Genome_id = Genome_id
        self.creature_name = creature_name
        self.segments = []

def process_file(filename):
    Genomes = {}
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split('#')
            creature_name = parts[3]
            Genome_id = parts[1]
            if Genome_id not in Genomes:
                Genomes[Genome_id] = Genome(Genome_id, creature_name)
            genome = Genomes[Genome_id]
            segments = parts[-1].split()
            current_segment = []
            for segment in segments:
                if segment.isdigit() and len(segment) == 4:
                    current_segment.append(segment)
                else:
                    if current_segment:
                        genome.segments.append(current_segment)
                    current_segment = []
            if current_segment:
                genome.segments.append(current_segment)
    return Genomes

def get_clusters_of_length_d_at_least_q_times(Genomes, d, q):
    segment_counts = {}

    for Genome_id, Genome in Genomes.items():
        for segment in Genome.segments:
            if len(segment) >= d:
                for i in range(len(segment) - d + 1):
                    sub_segment = segment[i:i+d]
                    segment_tuple = tuple(sorted(sub_segment))
                    if segment_tuple in segment_counts:
                        if Genome_id not in segment_counts[segment_tuple]:
                            segment_counts[segment_tuple][Genome_id] = (sub_segment, Genome.creature_name)
                    else:
                        segment_counts[segment_tuple] = {Genome_id: (sub_segment, Genome.creature_name)}

    filtered_clusters = {}
    for segment, genomes in segment_counts.items():
        if len(genomes) >= q:
            filtered_clusters[segment] = genomes
    top_clusters = dict(sorted(filtered_clusters.items(), key=lambda item: len(item[1]), reverse=True)[:5])
    return top_clusters

# Main code
q = 4
d = 2
while (d<5):
  #noa: why d<5?
  filename = 'cog_words_plasmid.txt'
  start_time = datetime.now()
  Genomes = process_file(filename)

  selected_lists = get_clusters_of_length_d_at_least_q_times(Genomes, d, q)
  timedelta = datetime.now() - start_time
  print(timedelta)
  print()
  #noa: why top 5?
  print(" "*30 + "Top 5 clusters of length", d, "appearing at least", q, "times:")
  print()
  for segment, cluster_info in selected_lists.items():
      print("Cluster:", segment)
      print("Occurrences:", len(cluster_info))
      print("Genome ID's and Creature Names:")
      for genome_id, (order, creature_name) in cluster_info.items():
          print(f"\tGenome ID: {genome_id}, Creature Name: {creature_name}, Order: {order}")
      print("First Instance of this Cluster:", next(iter(cluster_info.values()))[0])
      print("Information about the COG's:")
      for cog in segment:
        if cog in cog_dict:
          print("\t COG",cog, ":", cog_dict[cog])
        else:
          print("\t COG",cog, ":", "No Information was found for this COG")
      print("Statistics about the different orders we found:")
      word_counter = {}
      host_list = {}
      for value in cluster_info.values():
        word_string = ', '.join(value[0])
        word_counter[word_string] = word_counter.get(word_string, 0) + 1
        host_name = value[1]
        if word_string in host_list:
          host_list[word_string].append(host_name)
        else:
          host_list[word_string]= []
          host_list[word_string].append(host_name)
      for key, value in word_counter.items():
        print("\t [", key , "]:", value, "times out of", len(cluster_info), "which is", round(100*int(value)/len(cluster_info),2), "percent")
      print("Statistics about the Phylum:")
      for order in host_list.keys():
        print("\t Order: [", order , "] - Appears in these Phylum:")
        phylum_counter = {}
        total_hosts = len(host_list[order])
        for host in host_list[order]:
          curr_phylum = taxa_dict[host][1]
          phylum_counter[curr_phylum] = phylum_counter.get(curr_phylum, 0) + 1
        for key, value in phylum_counter.items():
          print("\t\t", key, "-", value, "times out of", total_hosts, ",which is", round(100*int(value)/total_hosts,2), "percent")
      print("Statistics about the Habitat:")
      for order in host_list.keys():
        print("\t Order: [", order , "] - Appears in these Habitats:")
        habitat_counter = {}
        total_hosts = len(host_list[order])
        for host in host_list[order]:
          curr_habitat = habitat_dict[host]
          habitat_counter[curr_habitat] = habitat_counter.get(curr_habitat, 0) + 1
        for key, value in habitat_counter.items():
          print("\t\t", key, "-", value, "times out of", total_hosts, ",which is", round(100*int(value)/total_hosts,2), "percent")
      print()
      print("*"*100)
  d = d+1
  print("*"*100)
  print("*"*100)

0:00:00.230021

                              Top 5 clusters of length 2 appearing at least 4 times:

Cluster: ('1192', '1475')
Occurrences: 309
Genome ID's and Creature Names:
	Genome ID: NC_009926, Creature Name: Acaryochloris_marina_MBIC11017_uid58167, Order: ['1192', '1475']
	Genome ID: NC_009927, Creature Name: Acaryochloris_marina_MBIC11017_uid58167, Order: ['1192', '1475']
	Genome ID: NC_009928, Creature Name: Acaryochloris_marina_MBIC11017_uid58167, Order: ['1192', '1475']
	Genome ID: NC_009930, Creature Name: Acaryochloris_marina_MBIC11017_uid58167, Order: ['1192', '1475']
	Genome ID: NC_009931, Creature Name: Acaryochloris_marina_MBIC11017_uid58167, Order: ['1192', '1475']
	Genome ID: NC_013210, Creature Name: Acetobacter_pasteurianus_IFO_3283_01_uid59279, Order: ['1192', '1475']
	Genome ID: NC_014641, Creature Name: Achromobacter_xylosoxidans_A8_uid59899, Order: ['1192', '1475']
	Genome ID: NC_014642, Creature Name: Achromobacter_xylosoxidans_A8_uid59899, Order: ['1192', '14